In [1]:
import pandas as pd
from astroquery.simbad import Simbad
from jwst import *

%load_ext autoreload
%autoreload 2
Simbad.add_votable_fields('flux(K)','sp')

max_bex_age = 1000 #This isn't the actually maximum age, but the maximum age above which there isn't much data. 

Directories to change depending on what computer you're on

In [2]:
figure_base_directory = "./Figures_F444W/"

In [3]:
computer = 'max_laptop'
connor = False
maxmb = True

if computer == 'connor':
    base_model_dir="/Users/connorvancil/Desktop/AstroResearch/Data/models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = None
elif computer == 'max_laptop': 
    base_model_dir="/Users/maxwellmb/Data/models/ATMO_2020_models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = '/Users/maxwellmb/Data/models/BEX/isochrones/BEX_evol_mags_-2_MH_0.00AC.dat'
elif computer == 'max_desktop':
    base_model_dir = "TBD"
    bex_file = None

Read in the Pearce Table

In [4]:
df = pd.read_csv('../allDiscAnalysisResults.csv')
data_arr = df[['Target','Distance_pc','Age_Myr','PearceMPlt_MJup', 'PearcePltApocentre_arcsec','DiscInnerEdgePeri_au','DiscOuterEdgeApo_au']].to_numpy()
target_dict={}
for row in data_arr:
    target_dict[row[0]]=row[1:]

Query all targets

In [5]:
targets=df['Target'].to_numpy()
queried_params=[];spts=[]
for target in targets:
    result_table = Simbad.query_object(target)
    spts.append(result_table['SP_TYPE'][0])
    queried_params.append([target,result_table['FLUX_K'][0]])

Force Spectral Types to those in ck04models

In [6]:
spts[40]='A5V'
spts[103]='A1V'
for i,spt in enumerate(spts):
    if spt not in lookuptable: 
        spts[i] = match_spt(spt)
[params.append(spts[i]) for i, params in  enumerate(queried_params)]

#adding another dictionary to hold these queried paramters in a more useful format
simbad_vals={}
for star in queried_params:
    simbad_vals[star[0]]=[star[1],star[2]]

Set up some configuration info

In [7]:
configs = {'f250m':  {'instrument':'nircam','instrument_mask':'mask335r'},
            'f300m': {'instrument':'nircam','instrument_mask':'mask335r'}, 
            'f356w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f410m': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f444w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f1140c':{'instrument':'miri',  'instrument_mask':None},
            'f1550c':{'instrument':'miri',  'instrument_mask':None},
            }
filter_list = list(configs.keys())
filter_list = ['f444w'] #For now only doing this filter because the bex filters only have this filter.

In [8]:
full_detection_list = []

for i, target in enumerate(targets):
    
    age = target_dict[target][1]
    distance = target_dict[target][0]
    comp_sep = target_dict[target][3] #Companion separation in arcseconds
    comp_mass = target_dict[target][2] #Companions mass in M_Jup

    disk_inner_edge = target_dict[target][4]/distance
    disk_outer_edge = target_dict[target][5]/distance
    
    kmag,spt = simbad_vals[target]
    
    mass_limits_list = []
    sep_list = []
    target_detection_array = []
    almost_detected_array = []

    print("Target {}: {}, {} Myr, {:.0f} pc, kmag={:.1f}".format(i,target,age,distance,kmag))

    if comp_sep < 0.4:
        print("\tCompanion separation for {} is < 0.4 arcseconds, skipping".format(target))
        continue

    for filter_name in filter_list:
        
        instrument = configs[filter_name]['instrument']
        instrument_mask = configs[filter_name]['instrument_mask']

        jwst_mag = get_jwst_mag(spt,kmag,instrument,filter_name,filter_dir="./",jwst_mask=instrument_mask,
                    norm_filter='bessel_k').value

        contrast_curves=read_contrast_curves()
        sep,companion_mags=companion_detection_limit(jwst_mag,filter_name,contrast_curves,plot=False);

        if comp_sep < np.min(sep):
            print("\tCompanion of {:.2f} for {} is closer than the IWA in {}, skipping".format(comp_sep,target,filter_name))
            continue

        model_dir = base_model_dir+model_instrument_directories.get(instrument.lower())
        
        if instrument.lower () == "nircam":
            model_dir+=model_nircam_mask_directories.get(instrument_mask.lower())

        if age < max_bex_age:
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,bex_model_filename=bex_file,
                            plot=True)
        else: 
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,
                            plot=False)

        if len(mass_limits) < len(sep):
            sep = sep[:len(mass_limits)]
        detected = detect_companion(sep,mass_limits,comp_sep,comp_mass)
        target_detection_array.append(detected)

        #What if we got 2x deeper?
        almost_detected = detect_companion(sep,mass_limits/2.,comp_sep,comp_mass)
        almost_detected_array.append(almost_detected)

        # print("Was the companion detected? {}".format())
        mass_limits_list.append(mass_limits)
        sep_list.append(sep)

    if np.any(target_detection_array):
        print("\tCOMPANION DETECTED!")
        full_detection_list.append(True)
        figure_directory = figure_base_directory+"/Detected/"
    elif np.any(almost_detected_array): 
        full_detection_list.append(False)
        figure_directory = figure_base_directory+"/Almost_Detected/"
    else: 
        full_detection_list.append(False)
        figure_directory = figure_base_directory+"/Non-Detected/"
    
    fig,axis = plt.subplots(1,1,figsize=(12,5))
    for i in range(len(mass_limits_list)):
        plt.semilogy(sep_list[i],mass_limits_list[i],label=filter_list[i])
    plt.title("{}, {} Myr, {} pc, kmag={:.1f}, Detected: {}".format(target,age,distance,kmag,full_detection_list[-1]))
    plt.scatter(comp_sep,comp_mass,label="Predicted Companion")

    ymin, ymax = axis.get_ylim()

    plt.fill_between([disk_inner_edge,disk_outer_edge],[ymin],y2=[ymax],color='k',alpha=0.1)
    plt.xlabel("Separation (arcseconds)")
    plt.ylabel(r"Mass Limit ($M_{Jup}$)")
    plt.legend()
    plt.savefig(figure_directory+target+".png",dpi=200)
    plt.close()

Target 0: HD113556, 15.0 Myr, 101 pc, kmag=7.2


/Users/maxwellmb/Dropbox (Personal)/Library/Python/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]


	COMPANION DETECTED!
Target 1: HD114082, 15.0 Myr, 95 pc, kmag=7.2
	Companion separation for HD114082 is < 0.4 arcseconds, skipping
Target 2: HD115617, 4000.0 Myr, 9 pc, kmag=3.0
Target 3: HD115820, 15.0 Myr, 116 pc, kmag=7.3
	Companion separation for HD115820 is < 0.4 arcseconds, skipping
Target 4: HD115892, 200.0 Myr, 18 pc, kmag=2.7
Target 5: HD120326, 15.5 Myr, 114 pc, kmag=7.5
	Companion separation for HD120326 is < 0.4 arcseconds, skipping
Target 6: HD121191, 15.5 Myr, 132 pc, kmag=7.7
	Companion separation for HD121191 is < 0.4 arcseconds, skipping
Target 7: HD121617, 15.5 Myr, 116 pc, kmag=7.2
	Companion separation for HD121617 is < 0.4 arcseconds, skipping
Target 8: HD123889, 16.0 Myr, 128 pc, kmag=7.7
	Companion separation for HD123889 is < 0.4 arcseconds, skipping
Target 9: HD125162, 300.0 Myr, 30 pc, kmag=3.9
	Companion separation for HD125162 is < 0.4 arcseconds, skipping
Target 10: HD125451, 1400.0 Myr, 26 pc, kmag=4.4
Target 11: HD125541, 16.0 Myr, 160 pc, kmag=8.1
	Comp

In [ ]:
len(mass_limits)

77